In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Users\\devenv\\SPARK\\spark-3.3.1-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession

In [4]:
# This from the course
spark = SparkSession.builder.appName('Ops').getOrCreate() # .builder not .Builder !! waht

In [5]:
# this is mashed together from the book + a medium article
# If I run this after running the above it will error
# import findspark
# findspark.init()
# findspark.find()
# import pyspark
# findspark.find()
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession
# conf = SparkConf().setAppName('Ops').setMaster('local')
# sc = SparkContext(conf=conf)
# spark = SparkSession(sc)

In [6]:
import os
os.chdir(r'C:\Shared Files\Udemy Files\Python-and-Spark-for-Big-Data-master\Spark_DataFrames')

In [7]:
os.getcwd()

'C:\\Shared Files\\Udemy Files\\Python-and-Spark-for-Big-Data-master\\Spark_DataFrames'

In [8]:
# df = spark.read.json('people.json') # this errors - spark does not seem to care about chdir()

In [9]:
# note need to escape the final \ even in a raw string
file_directory = r'C:\Shared Files\Udemy Files\Python-and-Spark-for-Big-Data-master\Spark_DataFrames\\'

In [10]:
df = spark.read.json(f'{file_directory}people.json')

In [11]:
df

DataFrame[age: bigint, name: string]

In [12]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [13]:
df.columns

['age', 'name']

In [14]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [15]:
df.name, df['name']

(Column<'name'>, Column<'name'>)

In [16]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [17]:
from pyspark.sql.types import (StringType, StructType,
                               IntegerType, StructField) # remember! brackets for new lines!

In [18]:
data_schema = [StructField("age", IntegerType(), True),
              StructField("name", StringType(), True)]

In [19]:
final_struct = StructType(data_schema)

In [20]:
print(df.schema)
print(final_struct)

StructType([StructField('age', LongType(), True), StructField('name', StringType(), True)])
StructType([StructField('age', IntegerType(), True), StructField('name', StringType(), True)])


In [21]:
df = spark.read.json(f'{file_directory}people.json', schema=final_struct)

In [22]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [23]:
df.schema == final_struct, df.schema is final_struct

(True, False)

In [24]:
df.schema

StructType([StructField('age', IntegerType(), True), StructField('name', StringType(), True)])

In [25]:
df.printSchema()#

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Dataframe basics part II

In [26]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [27]:
df.withColumn('new_age', df.age).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     30|
|  19| Justin|     19|
+----+-------+-------+



In [28]:
# df2 = spark.createDataFrame([1, 2, 3]) # TypeError: Can not infer schema for type: <class 'int'> "seriously?"
df2_fields = [StructField('age2', IntegerType(), False)]
df2_schema = StructType(df2_fields)
# the data arg 1 has to be passed as a list of tuples, or dicts, it can be a row object, or rdd or PD.DataFrame too
df2 = spark.createDataFrame([(10,), (20,), (30,)], schema=df2_schema)

In [29]:
df2.show()

+----+
|age2|
+----+
|  10|
|  20|
|  30|
+----+



In [30]:
df2_2 = spark.createDataFrame([(10,), (20,), (30,)],\
                               schema=StructType([StructField('age2', IntegerType(), False)])) # bit longwinded
df2_2

DataFrame[age2: int]

In [31]:
df2_3 = spark.createDataFrame([(10,), (20,), (30,)],\
                                 schema='age int') # much nicer can be long, short, double also 
df2_3

DataFrame[age: int]

In [32]:
df2_row = pyspark.sql.Row("age2") # for Row() if I just pass comma seperated stings those will be the column names

In [33]:
df2_row(1) # we can add a value like so but this does not seem like a good way to populate a df...

Row(age2=1)

In [34]:
# ok now were getting into it -REMEMBER!! range does not inlclude final number
df3 = spark.createDataFrame([(i,) for i in range(1,4)], StructType([StructField("age3", IntegerType(), True)]))

In [35]:
df3.show()

+----+
|age3|
+----+
|   1|
|   2|
|   3|
+----+



In [36]:
df2.age2

Column<'age2'>

In [37]:
# actually this is because you can only withColumns an expression of the calling df or a lit() literal
# df.withColumn("age2", df2.age2) # this may error as the structField is not the same on nulls
# df.withColumn('age3', df.age3).show() # nope cos this also errors...

# this looks like it needs some kind of index thing - this is not Pandas or Numpy inplace magic...
# https://stackoverflow.com/questions/42853778/add-a-column-from-another-dataframe

In [38]:
df4 = spark.createDataFrame([(i,) for i in range(1,4)], schema=StructType([StructField("age4", IntegerType(), True)]))

In [39]:
df.rdd.zip(df.rdd).collect()

[(Row(age=None, name='Michael'), Row(age=None, name='Michael')),
 (Row(age=30, name='Andy'), Row(age=30, name='Andy')),
 (Row(age=19, name='Justin'), Row(age=19, name='Justin'))]

In [41]:
# df.rdd.zip(df2.rdd).collect() # will error on Can only zip with RDD which has the same number of partitions

In [42]:
df.rdd.getNumPartitions(), df2.rdd.getNumPartitions(), df3.rdd.getNumPartitions(), df4.rdd.getNumPartitions()
# why the heck do df2-4 have 8 partitions?
# here - https://www.talend.com/blog/2018/03/05/intro-apache-spark-partitioning-need-know/ 
# it says There are many factors which affect partitioning choices like:
#     Available resources — Number of cores on which task can run on.
#     External data sources — Size of local collections, Cassandra table or HDFS file determine number of partitions.
#  note this one   Transformations used to derive RDD 
#                 — There are a number of rules to determine the number of partitions 
#                 when a RDD is derived from another RDD.
# so as we built the dfs in a different manner = different rdd partitons?

(1, 8, 8, 8)

In [43]:
df.rdd.repartition(8).collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [44]:
# this gives horrific stack trace error poss cos repartition does not retun an rdd but a partition thingy
# MapPartitionsRDD[194] at coalesce at NativeMethodAccessorImpl.java:0
# df.rdd.repartition(8).zip(df2.rdd).collect()

In [45]:
df_rdd = df.rdd.repartition(8)

In [47]:
# df_rdd.zip(df2.rdd).collect() # this errors on Can only zip RDDs with same number of elements in each partition

In [48]:
# let's flip that
df.rdd.zip(df2.rdd.repartition(1)).collect()
# this looks more interesting except the fist df is tuplized!

[(Row(age=None, name='Michael'), Row(age2=10)),
 (Row(age=30, name='Andy'), Row(age2=20)),
 (Row(age=19, name='Justin'), Row(age2=30))]

In [49]:
# trippy = schema is gone too
df_col_add = spark.createDataFrame(df.rdd.zip(df2.rdd.repartition(1)).collect())
df_col_add.show()
## Perhaps you could do a withColumn on this split col _1 make a new col and replace the old one
## and pass a new schema

+---------------+----+
|             _1|  _2|
+---------------+----+
|{null, Michael}|{10}|
|     {30, Andy}|{20}|
|   {19, Justin}|{30}|
+---------------+----+



the world seems to think we should join...<br>
this is an example of a trippy join with a complex condition from:<br>
https://stackoverflow.com/questions/52850603/pyspark-add-columns-to-dataframe-based-on-values-from-different-dataframe<br>
cond = (df_bb.cell == df_aa.cell1)|(df_bb.cell == df_aa.cell2)<br>
df_bb.join(df_aa, cond, how="full").withColumn("val1", when((col("id1")==col("id2")) & ((col("cell")==col("cell1"))|(col("cell")==col("cell2"))) & (col("nr")==1), 1).otherwise(0)).withColumn("val2", when(~(col("id1")==col("id2")) & ((col("cell")==col("cell1"))|(col("cell")==col("cell2"))) & (col("nr")==1), 1).otherwise(0)).show()

In [55]:
# join needs an index
# this don't seem to work AssertionError: col should be Column
# this is the whole thingy here: from the Docstring
#  The column expression must be an expression over this :class:`DataFrame`; attempting to add
#  a column from some other :class:`DataFrame` will raise an error.
# df2.withColumn('name2', spark.sparkContext.parallelize([('Michael',),('Andy',),('Justin',)])).show()

In [56]:
#et's look at the join syntax
df_x_2 = df.join(df2) # this gives us cartesian/cross join...
df_x_2.show()

+----+-------+----+
| age|   name|age2|
+----+-------+----+
|null|Michael|  10|
|  30|   Andy|  10|
|  19| Justin|  10|
|null|Michael|  20|
|  30|   Andy|  20|
|  19| Justin|  20|
|null|Michael|  30|
|  30|   Andy|  30|
|  19| Justin|  30|
+----+-------+----+



In [57]:
# row_number() mod? 3
# from pyspark.sql.functions import col, row_number
# from pyspark.sql.window import Window

#df_x_2.select(df_x_2.columns).row_number().over(Window.orderBy(col("name").desc()))
# AttributeError: 'DataFrame' object has no attribute 'row_number'

In [58]:
# attempt 2
# ref https://stackoverflow.com/questions/55690705/how-to-select-a-range-of-rows-from-a-dataframe-in-pyspark
from pyspark.sql.functions import lit, row_number, col
from pyspark.sql.window import Window

# we can set up a window object with one partition and no order and apply it later - v. python
w = Window().partitionBy(lit('a')).orderBy(lit('a'))
w_nm = Window().orderBy('name') # I prefer this
df_i = df.withColumn("row_num", row_number().over(w_nm)) # I've heard you can use literals in a withColumn call

In [59]:
df_i.show()

+----+-------+-------+
| age|   name|row_num|
+----+-------+-------+
|  30|   Andy|      1|
|  19| Justin|      2|
|null|Michael|      3|
+----+-------+-------+



In [60]:
df2_i = df2.withColumn("row_num", row_number().over(w)) # here I still use the arbitrary as this is just demoing
df2_i.show()

+----+-------+
|age2|row_num|
+----+-------+
|  10|      1|
|  20|      2|
|  30|      3|
+----+-------+



In [61]:
help(w)

Help on WindowSpec in module pyspark.sql.window object:

class WindowSpec(builtins.object)
 |  WindowSpec(jspec: py4j.java_gateway.JavaObject) -> None
 |  
 |  A window specification that defines the partitioning, ordering,
 |  and frame boundaries.
 |  
 |  Use the static methods in :class:`Window` to create a :class:`WindowSpec`.
 |  
 |  .. versionadded:: 1.4.0
 |  
 |  Methods defined here:
 |  
 |  __init__(self, jspec: py4j.java_gateway.JavaObject) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  orderBy(self, *cols: Union[ForwardRef('ColumnOrName'), List[ForwardRef('ColumnOrName_')]]) -> 'WindowSpec'
 |      Defines the ordering columns in a :class:`WindowSpec`.
 |      
 |      .. versionadded:: 1.4.0
 |      
 |      Parameters
 |      ----------
 |      cols : str, :class:`Column` or list
 |          names of columns or expressions
 |  
 |  partitionBy(self, *cols: Union[ForwardRef('ColumnOrName'), List[ForwardRef('ColumnOrName_')]]) -> 

In [62]:
# it's nice when it works
df_j_1 = df_i.join(df2_i, on="row_num")
df_j_1.drop("row_num").show()

+----+-------+----+
| age|   name|age2|
+----+-------+----+
|  30|   Andy|  10|
|  19| Justin|  20|
|null|Michael|  30|
+----+-------+----+



#### Play time! this from the Databricks docs
the Row() function looks like namedtuple<br>
The rows can contain different lengthed sequences of rows<br>
the resultant structure is much like a json

In [63]:
# import pyspark class Row from module sql
from pyspark.sql import *
# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)
employee5 = Employee('michael', 'jackson', 'no-reply@neverla.nd', 80000)

# Create the DepartmentWithEmployees instances from Departments and Employees
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee5, employee4])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

print(department1)
print(employee2)
print(departmentWithEmployees1.employees[0].email)
print(departmentWithEmployees4)

Row(id='123456', name='Computer Science')
Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000)
no-reply@berkeley.edu
Row(department=Row(id='901234', name='Indoor Recreation'), employees=[Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000), Row(firstName='matei', lastName=None, email='no-reply@waterloo.edu', salary=140000)])
